In [1]:
import sys, os
sys.path.append(os.pardir)

In [1]:
from data import CustomDataset
from tokenizers import Tokenizer
import torch

tokenizer = Tokenizer.from_file("../data/WMT14_Tokenizer.json")

# training_data = CustomDataset(tokenizer=tokenizer, src_path="../data/training/training_en.txt", tgt_path="../data/training/training_de.txt")
# torch.save(training_data, "training_custom_dataset.pt")
training_data = torch.load("training_custom_dataset.pt")

In [4]:
training_data.__getitem__(1)

[tensor([    1, 22038, 13119, 30603,  5056,  5311,  5140,  6483,  5056, 13320,
          4574,  5113,  5085,  6155, 15560, 21635,  6799,  3373,     2,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [8]:
import torch

In [9]:
torch.save(training_data, "training_custom_dataset.pt")

In [4]:
len(training_data.tgt)

4501509

In [ ]:
tokenizer.decode(training_data.src[0])

In [ ]:
tokenizer.decode(training_data.tgt[0])

In [ ]:
len(training_data.tgt)

In [ ]:
if (5 > 7) | (3 > 5) :
    print("kkk")

In [1]:
import torch

In [8]:
def lrate(step_num, d_model, warmup_steps):
    with torch.no_grad():
        step1 = torch.pow(torch.tensor(d_model),-0.5)
        step2 = torch.min(torch.tensor((torch.pow(torch.tensor(step_num),-0.5), step_num*torch.pow(torch.tensor(warmup_steps),-1.5))))
        learning_rate = step1*step2
    return learning_rate

In [28]:
lrate(0, 512, 4000)

tensor(0.)

In [30]:
1e-9

1e-09

In [32]:
torch.pow(torch.tensor(10.), -9)

tensor(1.0000e-09)

In [33]:
aaa = [1,2,3,4,5]

In [34]:
aaa[:-1]

[1, 2, 3, 4]